<a href="https://colab.research.google.com/github/k1-furuya/rstan001/blob/main/%E9%A0%BB%E5%BA%A6%E8%AB%96AB%E3%83%86%E3%82%B9%E3%83%88%2Ct%E6%A4%9C%E5%AE%9A.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

従来のWEBページAと、改善後のページBをランダムに表示するABテストを実施し、その効果を検証する。データがまだ少ない状況を想定する為、意図的にサンプルサイズは小さくしている。


In [29]:
import pandas as pd
# 使うデータの視覚化
data = {
    'ページ': ['従来のページ', '改善後のページ'],
    '表示数': [40, 25],
    'クリック数': [2, 2]
}
df = pd.DataFrame(data)

df

,ページ,表示数,クリック数
0,従来のページ,40,2
1,改善後のページ,25,2


In [30]:
import numpy as np
import scipy.stats as stats

In [31]:
# グループAの結果
n_a = 40  # 表示数
obs_a = 2  # ヒット数

# グループBの結果
n_b = 25  # 表示数
obs_b = 2  # ヒット数


In [32]:
# 各グループのコンバージョン率の推定値を計算

p_a = obs_a / n_a
p_b = obs_b / n_b


In [33]:
# コンバージョン率の差
p_diff = p_b - p_a


In [34]:
# 各グループのコンバージョン率の標準誤差（SE）を計算
se_a = np.sqrt(p_a * (1 - p_a) / n_a)
se_b = np.sqrt(p_b * (1 - p_b) / n_b)

In [35]:
# コンバージョン率の差の標準誤差を計算
se_diff = np.sqrt(se_a**2 + se_b**2)

In [36]:
# tスコアを計算
t_score = p_diff / se_diff


In [37]:
# 自由度を計算します。
# ここではウェルチのt検定の自由度の近似式を使用。
df = (se_diff**4) / ((se_a**4 / (n_a - 1)) + (se_b**4 / (n_b - 1)))

In [38]:
# p値を計算
p_value = stats.t.sf(np.abs(t_score), df) * 2  # 両側検定

p_value

0.6430499461438455

In [39]:
# 95%信頼区間を計算
# t分布を使用して、信頼区間の境界を計算
t_critical = stats.t.ppf(1 - 0.05 / 2, df)  # 95%信頼区間のt値
ci_lower = p_diff - t_critical * se_diff  # 信頼区間の下限
ci_upper = p_diff + t_critical * se_diff  # 信頼区間の上限

ci_lower, ci_upper

(-0.09962951043724405, 0.15962951043724405)

グループBのコンバージョン率がグループAよりも最大15.9%ポイント高く、最大9.9%ポイント低い可能性があることを示しています。 信頼区間にゼロが含まれているため、統計的に有意な差があるとは言えません。信頼区間の幅が広い（約25.8%ポイント）ことから、コンバージョン率の差に関する推定にはかなりの不確実性があることがわかります。

p値が0.643という結果は、ABテストで観察されたコンバージョン率の差について、統計的に有意な差がないと解釈できます。頻度論的アプローチでは、p値が一般的に設定される有意水準（ここでは0.05）よりも大きい場合、帰無仮説（ここではグループAとグループBの間にコンバージョン率の差がないという仮説）を棄却するには十分な証拠が得られていないと考えます。

言い換えると、p値0.643は、観察されたコンバージョン率の差が偶然によっても生じうる範囲内であるということを意味しています。そのため、このテストではグループAとグループBの間に**統計的に意味のある差があるとは言えないという結論**に至ります。

（ここの言葉の分かりにくさもポイントのひとつ）